In [1]:
import os
import sys
import pickle
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
sys.path.append(os.path.dirname(os.path.abspath('.')))
import configs
from datasets import TruthfulQADataset, DishonestQADataset, AmongUsDataset, RolePlayingDataset, RepEngDataset, ApolloProbeDataset
from probes import LinearProbe

from configs import config_phi4

In [2]:
dataset_name = 'DishonestQADataset'
config = config_phi4
model, tokenizer, device = None, None, 'cpu'
EXPT_NAME: str = "2025-02-01_phi_phi_100_games_v3"
dataset = eval(f"{dataset_name}")(config, model=model, tokenizer=tokenizer, device=device, expt_name=EXPT_NAME, test_split=0.2)
# dataset = eval(f"{dataset_name}")(config, model=model, tokenizer=tokenizer, device=device, test_split=0.2)

In [3]:
# dataset.populate_dataset(force_redo=False, just_load=True)
dataset.populate_dataset(force_redo=False)

Loading existing chunk from ./data/DishonestQA_phi4_acts/chunk_0.pkl


In [4]:
train_loader = dataset.get_train(batch_size=32, num_tokens=10, chunk_idx=0)

In [12]:
probe = LinearProbe(input_dim=dataset.activation_size, device=device)

print(f'Training probe on {len(train_loader)} batches and {len(train_loader.dataset)} samples.')

Training probe on 395 batches and 12640 samples.


In [15]:
probe.fit(train_loader, epochs=4)

ZeroDivisionError: integer division or modulo by zero

In [14]:
checkpoint_path = f'checkpoints/{dataset_name}_probe_{config["short_name"]}.pkl'
with open(checkpoint_path, 'wb') as f:
    pickle.dump(probe, f)
    print(f"Probe saved to {checkpoint_path}")

Probe saved to checkpoints/DishonestQADataset_probe_phi4.pkl
